In [1]:
import pandas as pd
import torch
from config2 import CFG, logging_conf
from lightgcn.datasets2 import *
from lightgcn.models2 import build, train
from lightgcn.utils import class2dict, get_logger

logger = get_logger(logging_conf)
use_cuda = torch.cuda.is_available() and CFG.use_cuda_if_available
device = torch.device("cuda" if use_cuda else "cpu")
print(device)


/opt/conda/envs/lightgcn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
train_data, valid_data, test_data, n_node, feature_input_dims = prepare_dataset(
        device, CFG.basepath, CFG.feature_names, verbose=CFG.loader_verbose, logger=logger.getChild("data")
)

2022-11-21 13:21:57,430 - data - INFO - Train Dataset Info
2022-11-21 13:21:57,431 - data - INFO -  * Num. Users    : 7442
2022-11-21 13:21:57,432 - data - INFO -  * Max. UserID   : 7441
2022-11-21 13:21:57,432 - data - INFO -  * Num. Items    : 9454
2022-11-21 13:21:57,433 - data - INFO -  * Num. Records  : 2475962
2022-11-21 13:21:57,434 - data - INFO - Test Dataset Info
2022-11-21 13:21:57,434 - data - INFO -  * Num. Users    : 744
2022-11-21 13:21:57,435 - data - INFO -  * Max. UserID   : 7439
2022-11-21 13:21:57,435 - data - INFO -  * Num. Items    : 444
2022-11-21 13:21:57,436 - data - INFO -  * Num. Records  : 744


In [3]:
train_data

{'edge': tensor([[    0,     0,     0,  ...,  7439,  7439,  7439],
         [12796, 12797, 12798,  ..., 11170, 11172, 11173]], device='cuda:0'),
 'label': tensor([1, 1, 1,  ..., 0, 1, 1], device='cuda:0'),
 'feature': tensor([[975, 975, 975,  ..., 713, 713, 713],
         [556, 557, 557,  ..., 719, 697, 697]], device='cuda:0')}

In [4]:
feature_input_dims

[1537, 912]

In [5]:
model = build(
        n_node,
        feature_input_dims,
        embedding_dim=CFG.embedding_dim,
        num_layers=CFG.num_layers,
        feature_embedding_dims=CFG.feature_embedding_dims,
        alpha=CFG.alpha,
        logger=logger.getChild("build"),
        **CFG.build_kwargs
    )
model.to(device)


2022-11-21 13:21:57,734 - build - INFO - No load model


LightGCN_context(
  (embedding): Embedding(16896, 64)
  (convs): ModuleList(
    (0): LGConv()
  )
  (feature_embedding): ModuleList(
    (0): Embedding(1537, 64)
    (1): Embedding(912, 64)
  )
  (comb_proj): Linear(in_features=128, out_features=64, bias=True)
  (predict_layer): Linear(in_features=192, out_features=1, bias=True)
)

In [6]:
train(
        model,
        train_data,
        n_epoch=CFG.n_epoch,
        learning_rate=CFG.learning_rate,
        use_wandb=CFG.user_wandb,
        weight=CFG.weight_basepath,
        logger=logger.getChild("train"),
    )

2022-11-21 13:21:57,826 - train - INFO - Training Started : n_epoch=20
2022-11-21 13:21:58,478 - train - INFO -  * In epoch 0001, loss=0.692, acc=0.552, AUC=0.549


Error: You must call wandb.init() before wandb.log()

In [8]:
train_data.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


In [28]:
def indexing_data(data, feature_names):
    idx = {}
    userid, itemid = (
        sorted(list(set(data.userID))),
        sorted(list(set(data.assessmentItemID))),
    )
    n_user, n_item = len(userid), len(itemid)

    userid_2_index = {v: i for i, v in enumerate(userid)}
    itemid_2_index = {v: i + n_user for i, v in enumerate(itemid)}
    idx['user_item'] = dict(userid_2_index, **itemid_2_index)
    
    feature_input_dims = []
    for feature_name in feature_names:
        featid = sorted(list(set(data[feature_name])))
        feature_input_dims.append(len(featid))
        idx[feature_name] = {v: i for i, v in enumerate(featid)}
    return idx

In [3]:
tmop = pd.Series([1,2,3])
tmop.to_list()

[1, 2, 3]

In [71]:
def process_data(data, id_2_index, device):
    edge, label, feature = [], [], []
    for user, item, acode in zip(data.userID, data.assessmentItemID, data.answerCode):
        uid, iid = id_2_index['user_item'][user], id_2_index['user_item'][item]
        edge.append([uid, iid])
        label.append(acode)

    for k, v in id_2_index.items():
        if k != 'user_item':
            feat = data[k].map(v).tolist()
            feature.append(feat)
    import pdb; pdb.set_trace()

    edge = torch.LongTensor(edge).T
    label = torch.LongTensor(label)
    feature = torch.LongTensor(feature)

    return dict(edge=edge.to(device), label=label.to(device), feature=feature.to(device))

In [72]:
tmp = process_data(train_data, id2index, device)

> /tmp/ipykernel_1603/1769348169.py(14)process_data()
     12     import pdb; pdb.set_trace()
     13 
---> 14     edge = torch.LongTensor(edge).T
     15     label = torch.LongTensor(label)
     16     feature = torch.LongTensor(feature)

0         975
1         975
2         975
3         975
4         975
         ... 
260108    780
260109    713
260110    713
260111    713
260112    713
Name: testId, Length: 2475962, dtype: int64
0         975
1         975
2         975
3         975
4         975
         ... 
260108    780
260109    713
260110    713
260111    713
260112    713
Name: testId, Length: 2475962, dtype: int64
tensor([[975, 975, 975,  ..., 713, 713, 713],
        [556, 557, 557,  ..., 719, 697, 697]])
tensor([[975, 975, 975,  ..., 713, 713, 713],
        [556, 557, 557,  ..., 719, 697, 697]])


In [64]:
tmp

{'edge': tensor([[    0,     0,     0,  ...,  7439,  7439,  7439],
         [12796, 12797, 12798,  ..., 11171, 11172, 11173]], device='cuda:0'),
 'label': tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0'),
 'feature': tensor([[975, 975, 975,  ..., 713, 713, 713],
         [556, 557, 557,  ..., 719, 697, 697]], device='cuda:0')}

In [17]:
train_data.drop(['userID', 'assessmentItemID', 'answerCode', 'Timestamp'], axis=1).values.T

array([['A060000001', 'A060000001', 'A060000001', ..., 'A040000130',
        'A040000130', 'A040000130'],
       [7224, 7225, 7225, ..., 8832, 8244, 8244]], dtype=object)

In [11]:
process_data(train_data, id2index, device)

{'edge': tensor([[    0,     0,     0,  ...,  7439,  7439,  7439],
         [12796, 12797, 12798,  ..., 11171, 11172, 11173]], device='cuda:0'),
 'label': tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0')}

In [3]:
train_data

{'edge': tensor([[    0,     0,     0,  ...,  7439,  7439,  7439],
         [12796, 12797, 12798,  ..., 11171, 11172, 11173]], device='cuda:0'),
 'label': tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0')}

In [47]:
tmp = torch.tensor([[[1,2,3], [4,5,6], [7,8,9], [10,11,12]],
              [[13,14,15], [16,17,18], [19,20,21], [22, 23, 24]]])
tmp2 = tmp + 20

In [52]:
torch.cat([tmp.view(8,3), tmp2.view(8,3)],1)

tensor([[ 1,  2,  3, 21, 22, 23],
        [ 4,  5,  6, 24, 25, 26],
        [ 7,  8,  9, 27, 28, 29],
        [10, 11, 12, 30, 31, 32],
        [13, 14, 15, 33, 34, 35],
        [16, 17, 18, 36, 37, 38],
        [19, 20, 21, 39, 40, 41],
        [22, 23, 24, 42, 43, 44]])

In [5]:
import numpy as np
tmp = torch.tensor(np.arange(10))

In [7]:
len(tmp)

10

In [10]:
for i in []:
    print(i)

In [9]:
tmp = pd.read_csv('../../data/train_data.csv')


<function lightgcn.models2.train(model, train_data, valid_data=None, n_epoch=100, learning_rate=0.01, use_wandb=False, weight=None, logger=None)>

In [10]:
def tmp(**kwargs):
    print(a)

In [11]:
def tmp2(**kwargs):
    tmp(**kwargs)

In [12]:
tmp2(a='d', b=1)

NameError: name 'a' is not defined